In [ ]:
# Mount Google Drive (for Colab)
from google.colab import drive
drive.mount('/content/drive/')
import os

os.chdir('/content/drive/MyDrive/Prof._Lee_Research/lib')


In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from denoising.config import *
from denoising.image_processing import load_tiff_stack, moving_average_subtraction, identify_particles
from denoising.particle_tracking import extract_roi, track_particles
from denoising.visualization import plot_frame, plot_tracking_results

# Set your file paths
dir_Path = '/content/drive/MyDrive/Prof._Lee_Research'
Path = f'{dir_Path}/Cell1_S1.tif'
ID = 'P1_C1_S1_11'
X, Y = 685, 660  # Starting coordinates
Sframe, Eframe = 1, 300

# Create output directory
stack_path, stack_file = os.path.split(Path)
saving_path = f'{stack_path}/Process/Particle {ID}/'
if not os.path.isdir(saving_path):
  os.makedirs(saving_path)
save_base_name = f'{saving_path}{os.path.splitext(stack_file)[0]}_Particle{ID}_frameRange{FRAME_RANGE}_X{X}_Y{Y}'

# Load and process the stack
print("Loading image stack...")
stack = load_tiff_stack(Path, Sframe, Eframe)
print(f"Stack loaded with shape: {stack.shape}")

# Perform background subtraction
print("Performing background subtraction...")
subtracted_matrix, subtracted_matrix_b, subtracted_matrix_d, subtracted_matrix_double = \
    moving_average_subtraction(stack, FRAME_RANGE)

# Identify particles
print("Identifying particles...")
particle_region, particle_region_b, particle_region_d = \
    identify_particles(subtracted_matrix, subtracted_matrix_b, subtracted_matrix_d, 
                      THRESH_STACK, PARTICLE_SIZE_THRESH)

# Extract ROI and track particles
print("Tracking particles...")
particle_region_roi = extract_roi(particle_region, Y, X, ROI)
particle_centroid_roi, updated_particle_region_roi, jump_frames = \
    track_particles(particle_region_roi, PARTICLE_DEFAULT_DIAMETER, JUMP_THRESH, MIN_AREA)

print(f"Tracking complete. {jump_frames} frames had jumps that were corrected.")

# Plot some results
print("Plotting sample frames...")
for i in range(0, min(10, stack.shape[0])):
    # Plot original frame ROI
    plot_frame(stack[i, Y-ROI:Y+ROI+1, X-ROI:X+ROI+1], f"Original Frame {i+1}")
    
    # Plot subtracted frame ROI
    plot_frame(subtracted_matrix[i, Y-ROI:Y+ROI+1, X-ROI:X+ROI+1], f"Subtracted Frame {i+1}")
    
    # Plot particle mask
    plot_frame(updated_particle_region_roi[i], f"Particle Mask Frame {i+1}")

# Save tracking results to disk
print("Saving results...")
np.save(f"{save_base_name}_centroids.npy", particle_centroid_roi)
np.save(f"{save_base_name}_masks.npy", updated_particle_region_roi)

print("Processing complete!")